In [3]:
import os
import re
import pandas as pd

In [4]:
import collections

In [5]:
import os
import argparse
import subprocess
from Bio import SeqIO

In [6]:
from itertools import repeat
from multiprocessing import Pool, freeze_support

In [69]:
InDir = "/mnt/d/Lab/16S-Taxa-Phlyo/data/M15"

In [6]:
!ls /mnt/d/Lab/16S-Taxa-Phlyo/data/M15 | wc -l

180


In [70]:
sampleID = []
single = {}
pair = {}
df = pd.DataFrame()
SampleList = collections.defaultdict(list)

for file in os.listdir(InDir):
    filePath = os.path.join(InDir, file)
    if file.endswith(".ab1") and os.path.getsize(filePath) > 0:
        info = file.split("__")
        sampleID.append(info[0])
        SampleList[info[0]].append(filePath)
        
for sample in set(sampleID):
    if len(SampleList[sample]) == 1:
        single[sample] = SampleList[sample]
    elif len(SampleList[sample]) == 2:
        pair[sample] = SampleList[sample]
        for end in pair[sample]:
            if "F__" in end:
                R1 = end
            elif "R__" in end:
                R2 = end
        df = df.append({"ID": sample, "R1":R1, "R2":R2},ignore_index=True)

In [71]:
single

{'DA_157': ['/mnt/d/Lab/16S-Taxa-Phlyo/data/M15/DA_157__M13R__F07_2004260514Q.ab1'],
 'DA_165': ['/mnt/d/Lab/16S-Taxa-Phlyo/data/M15/DA_165__M13R__C09_2004260527Q.ab1'],
 'DA_179': ['/mnt/d/Lab/16S-Taxa-Phlyo/data/M15/DA_179__M13F__D03_2004260520Q.ab1'],
 'DA_149': ['/mnt/d/Lab/16S-Taxa-Phlyo/data/M15/DA_149__M13F__E01_2004260508Q.ab1'],
 'DA_186': ['/mnt/d/Lab/16S-Taxa-Phlyo/data/M15/DA_186__M13R__E04_2004260539Q.ab1'],
 'DA_194': ['/mnt/d/Lab/16S-Taxa-Phlyo/data/M15/DA_194__M13R__E10_2004260545Q.ab1'],
 'DA_160': ['/mnt/d/Lab/16S-Taxa-Phlyo/data/M15/DA_160__M13R__A03_2004260557Q.ab1'],
 'DA_187': ['/mnt/d/Lab/16S-Taxa-Phlyo/data/M15/DA_187__M13F__G01_2004260540Q.ab1'],
 'DA_188': ['/mnt/d/Lab/16S-Taxa-Phlyo/data/M15/DA_188__M13R__A10_2004260541Q.ab1'],
 'DA_115': ['/mnt/d/Lab/16S-Taxa-Phlyo/data/M15/DA_115__M13R__H02_2004260535Q.ab1'],
 'DA_181': ['/mnt/d/Lab/16S-Taxa-Phlyo/data/M15/DA_181__M13R__D03_2004260560Q.ab1'],
 'DA_150': ['/mnt/d/Lab/16S-Taxa-Phlyo/data/M15/DA_150__M13R__A07

In [9]:
single.keys()

dict_keys(['DA_184', 'DA_202', 'DA_157', 'DA_181', 'DA_194', 'DA_187', 'DA_186', 'DA_160', 'DA_165', 'DA_150', 'DA_188', 'DA_179', 'DA_115', 'DA_149'])

In [10]:
single.values()

dict_values([['/mnt/d/Lab/16S-Taxa-Phlyo/data/M15/DA_184__M13F__H03_2004260524Q.ab1'], ['/mnt/d/Lab/16S-Taxa-Phlyo/data/M15/DA_202__M13F__B06_2004260550Q.ab1'], ['/mnt/d/Lab/16S-Taxa-Phlyo/data/M15/DA_157__M13R__F07_2004260514Q.ab1'], ['/mnt/d/Lab/16S-Taxa-Phlyo/data/M15/DA_181__M13R__D03_2004260560Q.ab1'], ['/mnt/d/Lab/16S-Taxa-Phlyo/data/M15/DA_194__M13R__E10_2004260545Q.ab1'], ['/mnt/d/Lab/16S-Taxa-Phlyo/data/M15/DA_187__M13F__G01_2004260540Q.ab1'], ['/mnt/d/Lab/16S-Taxa-Phlyo/data/M15/DA_186__M13R__E04_2004260539Q.ab1'], ['/mnt/d/Lab/16S-Taxa-Phlyo/data/M15/DA_160__M13R__A03_2004260557Q.ab1'], ['/mnt/d/Lab/16S-Taxa-Phlyo/data/M15/DA_165__M13R__C09_2004260527Q.ab1'], ['/mnt/d/Lab/16S-Taxa-Phlyo/data/M15/DA_150__M13R__A07_2004260509Q.ab1'], ['/mnt/d/Lab/16S-Taxa-Phlyo/data/M15/DA_188__M13R__A10_2004260541Q.ab1'], ['/mnt/d/Lab/16S-Taxa-Phlyo/data/M15/DA_179__M13F__D03_2004260520Q.ab1'], ['/mnt/d/Lab/16S-Taxa-Phlyo/data/M15/DA_115__M13R__H02_2004260535Q.ab1'], ['/mnt/d/Lab/16S-Taxa-Phl

In [16]:
single['DA_194']

['/mnt/d/Lab/16S-Taxa-Phlyo/data/M15/DA_194__M13R__E10_2004260545Q.ab1']

In [18]:
"R__" in single['DA_194'][0]

True

In [30]:
OutDir = "/mnt/d/Lab/16S-Taxa-Phlyo/result/M15"

In [72]:
def parseSingle(singleDict, OutDir):
    OutList = []
    singleDir = os.path.join(OutDir, "single", "fatsq")
    if os.path.exists(singleDir) == 0:
        os.makedirs(singleDir, 0o777, True)
    for ID in single.keys():
        if "F__" in single[ID][0]:
            OutList.append(os.path.join(singleDir, ID + "_R1.fastq"))
        elif "R__" in single[ID][0]:
            OutList.append(os.path.join(singleDir, ID + "_R2.fastq"))
    return OutList

In [73]:
OutList = parseSingle(single, OutDir)

In [74]:
OutList

['/mnt/d/Lab/16S-Taxa-Phlyo/result/M15/single/single/fatsq/DA_157_R2.fastq',
 '/mnt/d/Lab/16S-Taxa-Phlyo/result/M15/single/single/fatsq/DA_165_R2.fastq',
 '/mnt/d/Lab/16S-Taxa-Phlyo/result/M15/single/single/fatsq/DA_179_R1.fastq',
 '/mnt/d/Lab/16S-Taxa-Phlyo/result/M15/single/single/fatsq/DA_149_R1.fastq',
 '/mnt/d/Lab/16S-Taxa-Phlyo/result/M15/single/single/fatsq/DA_186_R2.fastq',
 '/mnt/d/Lab/16S-Taxa-Phlyo/result/M15/single/single/fatsq/DA_194_R2.fastq',
 '/mnt/d/Lab/16S-Taxa-Phlyo/result/M15/single/single/fatsq/DA_160_R2.fastq',
 '/mnt/d/Lab/16S-Taxa-Phlyo/result/M15/single/single/fatsq/DA_187_R1.fastq',
 '/mnt/d/Lab/16S-Taxa-Phlyo/result/M15/single/single/fatsq/DA_188_R2.fastq',
 '/mnt/d/Lab/16S-Taxa-Phlyo/result/M15/single/single/fatsq/DA_115_R2.fastq',
 '/mnt/d/Lab/16S-Taxa-Phlyo/result/M15/single/single/fatsq/DA_181_R2.fastq',
 '/mnt/d/Lab/16S-Taxa-Phlyo/result/M15/single/single/fatsq/DA_150_R2.fastq',
 '/mnt/d/Lab/16S-Taxa-Phlyo/result/M15/single/single/fatsq/DA_184_R1.fastq',

In [16]:
type(single.keys())

dict_keys

In [17]:
list(single.values())

[['/mnt/d/Lab/16S-Taxa-Phlyo/data/M15/DA_184__M13F__H03_2004260524Q.ab1'],
 ['/mnt/d/Lab/16S-Taxa-Phlyo/data/M15/DA_202__M13F__B06_2004260550Q.ab1'],
 ['/mnt/d/Lab/16S-Taxa-Phlyo/data/M15/DA_157__M13R__F07_2004260514Q.ab1'],
 ['/mnt/d/Lab/16S-Taxa-Phlyo/data/M15/DA_181__M13R__D03_2004260560Q.ab1'],
 ['/mnt/d/Lab/16S-Taxa-Phlyo/data/M15/DA_194__M13R__E10_2004260545Q.ab1'],
 ['/mnt/d/Lab/16S-Taxa-Phlyo/data/M15/DA_187__M13F__G01_2004260540Q.ab1'],
 ['/mnt/d/Lab/16S-Taxa-Phlyo/data/M15/DA_186__M13R__E04_2004260539Q.ab1'],
 ['/mnt/d/Lab/16S-Taxa-Phlyo/data/M15/DA_160__M13R__A03_2004260557Q.ab1'],
 ['/mnt/d/Lab/16S-Taxa-Phlyo/data/M15/DA_165__M13R__C09_2004260527Q.ab1'],
 ['/mnt/d/Lab/16S-Taxa-Phlyo/data/M15/DA_150__M13R__A07_2004260509Q.ab1'],
 ['/mnt/d/Lab/16S-Taxa-Phlyo/data/M15/DA_188__M13R__A10_2004260541Q.ab1'],
 ['/mnt/d/Lab/16S-Taxa-Phlyo/data/M15/DA_179__M13F__D03_2004260520Q.ab1'],
 ['/mnt/d/Lab/16S-Taxa-Phlyo/data/M15/DA_115__M13R__H02_2004260535Q.ab1'],
 ['/mnt/d/Lab/16S-Taxa-Ph

In [21]:
output = []
def reemovNestings(l): 
    for i in l: 
        if type(i) == list: 
            print(type(i))
            reemovNestings(i) 
        else: 
            output.append(i)
    return output

In [22]:
output = reemovNestings(list(single.values()))

<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>


In [23]:
output

['/mnt/d/Lab/16S-Taxa-Phlyo/data/M15/DA_184__M13F__H03_2004260524Q.ab1',
 '/mnt/d/Lab/16S-Taxa-Phlyo/data/M15/DA_202__M13F__B06_2004260550Q.ab1',
 '/mnt/d/Lab/16S-Taxa-Phlyo/data/M15/DA_157__M13R__F07_2004260514Q.ab1',
 '/mnt/d/Lab/16S-Taxa-Phlyo/data/M15/DA_181__M13R__D03_2004260560Q.ab1',
 '/mnt/d/Lab/16S-Taxa-Phlyo/data/M15/DA_194__M13R__E10_2004260545Q.ab1',
 '/mnt/d/Lab/16S-Taxa-Phlyo/data/M15/DA_187__M13F__G01_2004260540Q.ab1',
 '/mnt/d/Lab/16S-Taxa-Phlyo/data/M15/DA_186__M13R__E04_2004260539Q.ab1',
 '/mnt/d/Lab/16S-Taxa-Phlyo/data/M15/DA_160__M13R__A03_2004260557Q.ab1',
 '/mnt/d/Lab/16S-Taxa-Phlyo/data/M15/DA_165__M13R__C09_2004260527Q.ab1',
 '/mnt/d/Lab/16S-Taxa-Phlyo/data/M15/DA_150__M13R__A07_2004260509Q.ab1',
 '/mnt/d/Lab/16S-Taxa-Phlyo/data/M15/DA_188__M13R__A10_2004260541Q.ab1',
 '/mnt/d/Lab/16S-Taxa-Phlyo/data/M15/DA_179__M13F__D03_2004260520Q.ab1',
 '/mnt/d/Lab/16S-Taxa-Phlyo/data/M15/DA_115__M13R__H02_2004260535Q.ab1',
 '/mnt/d/Lab/16S-Taxa-Phlyo/data/M15/DA_149__M13F__

In [24]:
singleList = list(single.values())

In [25]:
singleList = [item for sublist in singleList for item in sublist]

In [75]:
singleList = [item for sublist in list(single.values()) for item in sublist]

In [76]:
singleList

['/mnt/d/Lab/16S-Taxa-Phlyo/data/M15/DA_157__M13R__F07_2004260514Q.ab1',
 '/mnt/d/Lab/16S-Taxa-Phlyo/data/M15/DA_165__M13R__C09_2004260527Q.ab1',
 '/mnt/d/Lab/16S-Taxa-Phlyo/data/M15/DA_179__M13F__D03_2004260520Q.ab1',
 '/mnt/d/Lab/16S-Taxa-Phlyo/data/M15/DA_149__M13F__E01_2004260508Q.ab1',
 '/mnt/d/Lab/16S-Taxa-Phlyo/data/M15/DA_186__M13R__E04_2004260539Q.ab1',
 '/mnt/d/Lab/16S-Taxa-Phlyo/data/M15/DA_194__M13R__E10_2004260545Q.ab1',
 '/mnt/d/Lab/16S-Taxa-Phlyo/data/M15/DA_160__M13R__A03_2004260557Q.ab1',
 '/mnt/d/Lab/16S-Taxa-Phlyo/data/M15/DA_187__M13F__G01_2004260540Q.ab1',
 '/mnt/d/Lab/16S-Taxa-Phlyo/data/M15/DA_188__M13R__A10_2004260541Q.ab1',
 '/mnt/d/Lab/16S-Taxa-Phlyo/data/M15/DA_115__M13R__H02_2004260535Q.ab1',
 '/mnt/d/Lab/16S-Taxa-Phlyo/data/M15/DA_181__M13R__D03_2004260560Q.ab1',
 '/mnt/d/Lab/16S-Taxa-Phlyo/data/M15/DA_150__M13R__A07_2004260509Q.ab1',
 '/mnt/d/Lab/16S-Taxa-Phlyo/data/M15/DA_184__M13F__H03_2004260524Q.ab1',
 '/mnt/d/Lab/16S-Taxa-Phlyo/data/M15/DA_202__M13F__

In [67]:
list(single.keys())

['DA_194',
 'DA_202',
 'DA_186',
 'DA_157',
 'DA_165',
 'DA_149',
 'DA_181',
 'DA_160',
 'DA_115',
 'DA_150',
 'DA_179',
 'DA_184',
 'DA_188',
 'DA_187']

In [36]:
ABI2FastqParallel(singleList, OutList)

In [9]:
OutDir = "/mnt/d/Lab/16S-Taxa-Phlyo/result/M15/single"

In [45]:
OutList

['/mnt/d/Lab/16S-Taxa-Phlyo/result/M15/single/fatsq/DA_184_R1.fastq',
 '/mnt/d/Lab/16S-Taxa-Phlyo/result/M15/single/fatsq/DA_202_R1.fastq',
 '/mnt/d/Lab/16S-Taxa-Phlyo/result/M15/single/fatsq/DA_157_R2.fastq',
 '/mnt/d/Lab/16S-Taxa-Phlyo/result/M15/single/fatsq/DA_181_R2.fastq',
 '/mnt/d/Lab/16S-Taxa-Phlyo/result/M15/single/fatsq/DA_194_R2.fastq',
 '/mnt/d/Lab/16S-Taxa-Phlyo/result/M15/single/fatsq/DA_187_R1.fastq',
 '/mnt/d/Lab/16S-Taxa-Phlyo/result/M15/single/fatsq/DA_186_R2.fastq',
 '/mnt/d/Lab/16S-Taxa-Phlyo/result/M15/single/fatsq/DA_160_R2.fastq',
 '/mnt/d/Lab/16S-Taxa-Phlyo/result/M15/single/fatsq/DA_165_R2.fastq',
 '/mnt/d/Lab/16S-Taxa-Phlyo/result/M15/single/fatsq/DA_150_R2.fastq',
 '/mnt/d/Lab/16S-Taxa-Phlyo/result/M15/single/fatsq/DA_188_R2.fastq',
 '/mnt/d/Lab/16S-Taxa-Phlyo/result/M15/single/fatsq/DA_179_R1.fastq',
 '/mnt/d/Lab/16S-Taxa-Phlyo/result/M15/single/fatsq/DA_115_R2.fastq',
 '/mnt/d/Lab/16S-Taxa-Phlyo/result/M15/single/fatsq/DA_149_R1.fastq']

In [46]:
os.path.split(OutList[0])

('/mnt/d/Lab/16S-Taxa-Phlyo/result/M15/single/fatsq', 'DA_184_R1.fastq')

fastp -i /mnt/d/Lab/16S-Taxa-Phlyo/data/M15/DA_170__M13F__G04_2004260531Q.ab1 -o /mnt/d/Lab/16S-Taxa-Phlyo/result/M15/single/trim/DA_184_trim.fastq --trim_front 30 --max_len 750 --cut_front --cut_tail --cut_window_size 20 --cut_mean_quality 30 --html /mnt/d/Lab/16S-Taxa-Phlyo/result/M15/single/fastp/DA_184.html --report_title DA_184-fastq-merge-report

In [59]:
RunFastpSingleParallel(OutList, OutDir)

In [83]:
InDir = "/mnt/d/Lab/16S-Taxa-Phlyo/result/M15/single/trim"

In [84]:
def parseFastqSingle(InDir, OutDir):
    allSeq = []
    fastaFileList = []
    fastaDir = os.path.join(OutDir, "fasta")
    if os.path.exists(fastaDir) == 0:
        os.makedirs(fastaDir, 0o777, True)
    for file in os.listdir(InDir):
        if file.endswith(".fastq") and os.path.getsize(os.path.join(InDir, file)) > 0:
            for seq in SeqIO.parse(os.path.join(InDir, file), "fastq"):
                seq.id = file.replace(".fastq", "")
                seq.name = ""
                seq.description = ""
                if "_R2" in seq.id:
                    seq.seq = seq.seq.reverse_complement()
                allSeq.append(seq)
                fastaFile = os.path.join(fastaDir, file.replace(".fastq", ".fasta"))
                fastaFileList.append(fastaFile)
                SeqIO.write(seq, fastaFile, "fasta")
    SeqIO.write(allSeq, os.path.join(OutDir, "single.fasta"), "fasta")
    return fastaFileList

In [85]:
fastaFileList = parseFastqSingle(InDir, OutDir)

In [86]:
OutDir

'/mnt/d/Lab/16S-Taxa-Phlyo/result/M15/single'

In [87]:
fastaFileList

['/mnt/d/Lab/16S-Taxa-Phlyo/result/M15/single/fasta/DA_115_R2.fasta',
 '/mnt/d/Lab/16S-Taxa-Phlyo/result/M15/single/fasta/DA_149_R1.fasta',
 '/mnt/d/Lab/16S-Taxa-Phlyo/result/M15/single/fasta/DA_150_R2.fasta',
 '/mnt/d/Lab/16S-Taxa-Phlyo/result/M15/single/fasta/DA_157_R2.fasta',
 '/mnt/d/Lab/16S-Taxa-Phlyo/result/M15/single/fasta/DA_160_R2.fasta',
 '/mnt/d/Lab/16S-Taxa-Phlyo/result/M15/single/fasta/DA_165_R2.fasta',
 '/mnt/d/Lab/16S-Taxa-Phlyo/result/M15/single/fasta/DA_179_R1.fasta',
 '/mnt/d/Lab/16S-Taxa-Phlyo/result/M15/single/fasta/DA_181_R2.fasta',
 '/mnt/d/Lab/16S-Taxa-Phlyo/result/M15/single/fasta/DA_184_R1.fasta',
 '/mnt/d/Lab/16S-Taxa-Phlyo/result/M15/single/fasta/DA_186_R2.fasta',
 '/mnt/d/Lab/16S-Taxa-Phlyo/result/M15/single/fasta/DA_187_R1.fasta',
 '/mnt/d/Lab/16S-Taxa-Phlyo/result/M15/single/fasta/DA_188_R2.fasta',
 '/mnt/d/Lab/16S-Taxa-Phlyo/result/M15/single/fasta/DA_194_R2.fasta',
 '/mnt/d/Lab/16S-Taxa-Phlyo/result/M15/single/fasta/DA_202_R1.fasta']

In [12]:
db = "/mnt/d/Lab/16S-Taxa-Phlyo/database/SILVA_138.1_SSURef_NR99_tax_silva.fasta"

In [13]:
def RunBlastnParallel(fastaList, db, jobs, threads, OutDir):
    pool = Pool(processes=jobs)
    pool.starmap(RunBlastn, zip(fastaList, repeat(db), repeat(threads), repeat(OutDir)))
    pool.close()
    pool.join()
    pool.terminate()

def RunBlastn(fasta, db, threads, OutDir):
    blastDir = os.path.join(OutDir, "blast")
    if os.path.exists(blastDir) == 0:
        os.makedirs(blastDir, 0o777, True)
    OutFile = os.path.join(blastDir, os.path.split(fasta)[1].replace(".fasta", "") + "_blast.tsv")
    cmd = "blastn -query " + fasta  + " -out " + OutFile + " -evalue 1.0 -max_target_seqs 1 -outfmt 6 -db " + db + " -num_threads " + str(threads) 
    print(cmd)
    subprocess.call(cmd, shell=True)

In [16]:
blastDir = os.path.join(OutDir, "blast")
if os.path.exists(blastDir) == 0:
    os.makedirs(blastDir, 0o777, True)

In [14]:
OutDir = "/mnt/d/Lab/16S-Taxa-Phlyo/result/M15/single"

In [15]:
RunBlastnParallel(fastaFileList, db, 2, 4, OutDir)

blastn -query /mnt/d/Lab/16S-Taxa-Phlyo/result/M15/single/fasta/DA_115_R2.fasta -out /mnt/d/Lab/16S-Taxa-Phlyo/result/M15/single/blast/DA_115_R2_blast.tsv -evalue 1.0 -max_target_seqs 1 -outfmt 6 -db /mnt/d/Lab/16S-Taxa-Phlyo/database/SILVA_138.1_SSURef_NR99_tax_silva.fasta -num_threads 4blastn -query /mnt/d/Lab/16S-Taxa-Phlyo/result/M15/single/fasta/DA_150_R2.fasta -out /mnt/d/Lab/16S-Taxa-Phlyo/result/M15/single/blast/DA_150_R2_blast.tsv -evalue 1.0 -max_target_seqs 1 -outfmt 6 -db /mnt/d/Lab/16S-Taxa-Phlyo/database/SILVA_138.1_SSURef_NR99_tax_silva.fasta -num_threads 4

blastn -query /mnt/d/Lab/16S-Taxa-Phlyo/result/M15/single/fasta/DA_149_R1.fasta -out /mnt/d/Lab/16S-Taxa-Phlyo/result/M15/single/blast/DA_149_R1_blast.tsv -evalue 1.0 -max_target_seqs 1 -outfmt 6 -db /mnt/d/Lab/16S-Taxa-Phlyo/database/SILVA_138.1_SSURef_NR99_tax_silva.fasta -num_threads 4
blastn -query /mnt/d/Lab/16S-Taxa-Phlyo/result/M15/single/fasta/DA_157_R2.fasta -out /mnt/d/Lab/16S-Taxa-Phlyo/result/M15/single/b

In [23]:
columns = ["qseqid", "sseqid", "pident", "length", "mismatch", "gapopen", "qstart", "qend", "sstart", "send", "evalue", "bitscore"]

In [33]:
def parseIndex(blastDir, OutDir):
    taxaList = []
    df = pd.DataFrame()
    out = pd.DataFrame()
    for file in os.listdir(blastDir):
        #print(file)
        if file.endswith(".tsv") and os.path.getsize(os.path.join(blastDir, file)) > 0:
            df1 = pd.read_table(os.path.join(blastDir, file), header = None)
            df1.columns = ["qseqid", "sseqid", "pident", "length", "mismatch", "gapopen", "qstart", "qend", "sstart", "send", "evalue", "bitscore"]
            df = df.append(df1)
    df = df.reset_index()
    df2 = pd.read_table(indexPath)
    index = dict(zip(df2["ID"], df2["Taxa"]))
    for i in range(len(df)):
        taxa = index[df["sseqid"][i]]
        taxaList.append(taxa)
    out["taxa"] = taxaList
    out["qseqid"] = df["qseqid"]
    out["pident"] = df["pident"]
    out["length"] = df["length"]
    return out

In [34]:
df1

,qseqid,sseqid,pident,length,mismatch,gapopen,qstart,qend,sstart,send,evalue,bitscore
0,DA_202_R1,AB506222.1.1514,99.867,750,1,0,1,750,58,807,0.0,1380


In [51]:
df = df.reset_index()

In [52]:
df

,index,qseqid,sseqid,pident,length,mismatch,gapopen,qstart,qend,sstart,send,evalue,bitscore
0,0,DA_115_R2,HQ779616.1.1444,100.000,750,0,0,1,750,659,1408,0.0,1386
1,0,DA_149_R1,FJ684727.1.1437,99.867,750,1,0,1,750,59,808,0.0,1380
2,0,DA_150_R2,HQ785168.1.1452,99.867,750,1,0,1,750,659,1408,0.0,1380
3,0,DA_157_R2,CDZP01049187.10668.12177,99.867,750,1,0,1,750,684,1433,0.0,1380
4,0,DA_160_R2,AGZQ01000017.129.1643,99.600,750,3,0,1,750,689,1438,0.0,1369
5,0,DA_165_R2,CCYG01000338.190.1701,99.067,750,7,0,1,750,690,1439,0.0,1347
6,0,DA_179_R1,JQ072766.1.1406,99.734,751,1,1,1,750,8,758,0.0,1375
7,0,DA_181_R2,CP022754.1214557.1216096,99.733,750,2,0,1,750,695,1444,0.0,1375
8,0,DA_184_R1,FLKM01000009.1335473.1336974,99.733,750,2,0,1,750,68,817,0.0,1375
9,0,DA_186_R2,GU562446.1.1426,99.867,750,1,0,1,750,624,1373,0.0,1380


In [37]:
df2 = pd.read_table("silva-138-99-index.tsv")

In [38]:
df2

,ID,Taxa
0,AY846379.1.1791,Eukaryota;Archaeplastida;Chloroplastida;Chloro...
1,AB001445.1.1538,Bacteria;Proteobacteria;Gammaproteobacteria;Ps...
2,AY929368.1.1768,Eukaryota;Archaeplastida;Chloroplastida;Charop...
3,KM209255.204.1909,Bacteria;Proteobacteria;Gammaproteobacteria;En...
4,AY955002.1.1727,Eukaryota;Archaeplastida;Chloroplastida;Chloro...
...,...,...
510503,MVBC01000034.3.1520,Bacteria;Bacteroidota;Bacteroidia;Cytophagales...
510504,JQIO01000430.1097739.1099039,Bacteria;Cyanobacteria;Cyanobacteriia;Chloropl...
510505,JQHZ01065714.557.1975,Bacteria;Cyanobacteria;Cyanobacteriia;Chloropl...
510506,JQIN01002109.43002.44826,Eukaryota;Archaeplastida;Chloroplastida;Charop...


In [39]:
index = dict(zip(df2["ID"], df2["Taxa"]))

In [41]:
index["AY846379.1.1791"]

'Eukaryota;Archaeplastida;Chloroplastida;Chlorophyta;Chlorophyceae;Sphaeropleales;Monoraphidium;Monoraphidium sp. Itas 9/21 14-6w'

In [55]:
df["sseqid"][0]

'HQ779616.1.1444'

In [44]:
len(df)

14

In [60]:
out = pd.DataFrame()
taxaList = []
for i in range(len(df)):
    taxa = index[df["sseqid"][i]]
    taxaList.append(taxa)
out["taxa"] = taxaList
out["qseqid"] = df["qseqid"]
out["pident"] = df["pident"]
out["length"] = df["length"]

In [61]:
out

,taxa,qseqid,pident,length
0,Bacteria;Firmicutes;Clostridia;Lachnospirales;...,DA_115_R2,100.000,750
1,Bacteria;Firmicutes;Clostridia;Lachnospirales;...,DA_149_R1,99.867,750
2,Bacteria;Bacteroidota;Bacteroidia;Bacteroidale...,DA_150_R2,99.867,750
3,Bacteria;Firmicutes;Clostridia;Oscillospirales...,DA_157_R2,99.867,750
4,Bacteria;Bacteroidota;Bacteroidia;Bacteroidale...,DA_160_R2,99.600,750
5,Bacteria;Firmicutes;Clostridia;Lachnospirales;...,DA_165_R2,99.067,750
6,Bacteria;Firmicutes;Bacilli;Lactobacillales;La...,DA_179_R1,99.734,751
7,Bacteria;Bacteroidota;Bacteroidia;Bacteroidale...,DA_181_R2,99.733,750
8,Bacteria;Firmicutes;Clostridia;Peptostreptococ...,DA_184_R1,99.733,750
9,Bacteria;Firmicutes;Clostridia;Oscillospirales...,DA_186_R2,99.867,750


In [27]:
def ABI2Fastq(abi, fastq):
    SeqIO.convert(abi, "abi", fastq, "fastq")

In [28]:
def ABI2FastqParallel(abiList, fastqList):
    pool = Pool(processes = 4)
    pool.starmap(ABI2Fastq, zip(abiList, fastqList))
    pool.close()
    pool.join()
    pool.terminate()

In [62]:
def RunFastpSingle(single, OutDir):
    prefix = os.path.split(single)[1].replace(".fastq", "")
    trimDir = os.path.join(OutDir, "trim")
    fastpDir = os.path.join(OutDir, "fastp")
    if os.path.exists(OutDir) == 0:
        os.makedirs(OutDir, 0o777, True)
    if os.path.exists(trimDir) == 0:
        os.makedirs(trimDir, 0o777, True)
    if os.path.exists(fastpDir) == 0:
        os.makedirs(fastpDir, 0o777, True)        
    cmd = "fastp -i " + single + " -o " + os.path.join(trimDir, prefix + ".fastq")  + \
    " --trim_front1 30 --cut_front --cut_tail --cut_window_size 20 --cut_mean_quality 30" + \
    " --html " + os.path.join(fastpDir, prefix + ".html") + " --report_title " + prefix + "-fastq-merge-report"
    #print(cmd)
    subprocess.call(cmd, shell=True)

In [56]:
def RunFastpSingleParallel(singleList, OutDir):
    pool = Pool(processes = 4)
    pool.starmap(RunFastpSingle, zip(singleList, repeat(OutDir)))
    pool.close()
    pool.join()
    pool.terminate()

反向的需要reverse回来！